In [4]:
import requests
import bs4 as bs
import tqdm
import pandas as pd

from src import *

%autoreload 2

# Data generation, identification of data sources

Initially it was suggested to  mine text from schools websites.  Later we've found an additional data source: the government open data, including Ofsted statistics. This comes in several CSV databases that can be downloaded from two sites:

https://www.compare-school-performance.service.gov.uk/download-data 

https://www.gov.uk/government/statistical-data-sets/monthly-management-information-ofsteds-school-inspections-outcomes 

The first link leads to CSV files with the 2015-2016 database with much of info that
can be obtained manually by surfing the site (www.compare-school-performance.service.gov.uk). The second link leads to the Ofsted public data (updated monthly). These CSV file is up to date (31 January 2017) and contains the overall Ofsted ratings for ~22K UK schools. 

#####  Scraping schools websites

Web scraping schools websites can be done using the functions in ``web_scraping.py``. 

``get_webpage_info`` : is the funciton that scrapes school websites given their URLs. This function has been tested with hundreds of schools websites. It handles pure HTML pages and JavaScript pages using a combination of libraries (requests, BeautifulSoup, Selenium, PhantomJS). Knowledge on HTML tags was needed to pinpoint the text from the 'about us' or 'ethos' webpages. Also many Python string manipulation tricks are used when matching the website content with a collection of interesting KEYWORDS. Some websites cannot be mined when they are down, non responsive or have a structure that we cannot parse. We expect to be able to mine ~70 percent of the total number of UK schools websites (~22K).

``get_webpage_text_from_url_list`` : we use this function to  grab the 'about-us' or 'ethos' text from a list of school URLs. A first big problem is obtaining the list of the school websites, which unfortunately is not available in any of the government databases.

#####  Mining the URLs

The task of retrieving the schools URLs was solved by scraping the government site:
https://www.compare-school-performance.service.gov.uk/. It is an online database where we can manually find schools according to different criteria. It displays metadata including the Ofsted rating and the actual schools website. We can automatize this procedure in two ways:

**I)** Directly scraping queries from https://www.compare-school-performance.service.gov.uk/
usign the functions: 
``get_schools_urls``, ``get_schools_metadata`` and ``generate_schools_db``.
We cannot find a way to have in a single query all UK schools at once, so this approach is semi-automatic. We could filter by location and obtain a few hundred schools in one batch.

**II)** Using the government open databases to obtain a list of URNs for all the schools in UK. With a school URN we can access the school entry in https://www.compare-school-performance.service.gov.uk/. This page is then scraped in order to obtain the actual school websites and later the mined text using ``get_schools_urls`` and ``get_webpage_text_from_url_list``

We'll use the second method. We can obtain ~20K UK schools with their Ofsted ratings and a lot of different useful information: school location, type, level, gender, faith denomination, number of pupils and teachers, pupils ages, etc.

#  Prototyping the web scraping code

**NOTE:** The  prototype was moved to the *web_scraping.py* module. A longer list of keywords is used.

###  Exploring Requests and BeautifulSoup

Randomly chosen  school website.

In [55]:
url = 'http://www.sohoparish.co.uk/'

In [411]:
from IPython.display import HTML

HTML('<iframe src='+url+' width=700 height=500></iframe>')

Raw data.

In [58]:
page = requests.get(url)
content = page.content
page.close()

In [74]:
print content[:500]

<!DOCTYPE html>
<html xmlns="http://www.w3.org/1999/xhtml">
<head id="cm_ctl00_templateHead"><script src="//fluencycontent-schoolwebsite.netdna-ssl.com/Releases/3764.0/Scripts/jquery.min.js" type="text/javascript"></script><meta http-equiv="Content-Type" content="text/html; charset=utf-8" /><meta name="description" content="Visit our website regularly for admissions guidelines, pupil resources, latest news stories, upcoming events and important notices." /><meta name="keywords" content="School 


http://web.stanford.edu/~zlotnick/TextAsData/Web_Scraping_with_Beautiful_Soup.html

http://chrisalbon.com/python/beautiful_soup_html_basics.html

http://youkilljohnny.blogspot.be/2014/03/beautifulsoup-cheat-sheet-parse-html-by.html

In [84]:
soup = bs.BeautifulSoup(content, 'lxml')#'html.parser')

The soup object contains the parsed HTML in the original raw document.


In [85]:
print soup.prettify()[0:500]

<!DOCTYPE html>
<html xmlns="http://www.w3.org/1999/xhtml">
 <head id="cm_ctl00_templateHead">
  <script src="//fluencycontent-schoolwebsite.netdna-ssl.com/Releases/3764.0/Scripts/jquery.min.js" type="text/javascript">
  </script>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <meta content="Visit our website regularly for admissions guidelines, pupil resources, latest news stories, upcoming events and important notices." name="description"/>
  <meta content="School web


In [90]:
soup.title

<title>
	Home | Soho Parish Primary
</title>

In [93]:
soup.p # first paragraph tag

<p><span style="background-color: transparent; line-height: 22px;">Soho Parish is a small school situated in the heart of London. We are a Church of England school and our values are firmly rooted in Christian principles. We have no faith criteria for entry, however, and serve our local community.</span></p>

In [94]:
soup.a # first link tag

<a href="mailto:office@sohoparish.co.uk">office@sohoparish.co.uk</a>

Tags:  
https://developer.mozilla.org/en-US/docs/Web/HTML/Element

https://www.w3schools.com/TAgs/

In [143]:
soup.findAll('a', text='About') # all links with text "about"

[<a class="rmLink rmRootLink" href="Heads-Welcome"><span class="rmText">About</span></a>]

In [151]:
interesting_text = ['About', 'about', 'about us', 'About us']

link_segment = soup.findAll('a', text=interesting_text)[0].attrs['href']
print link_segment

Heads-Welcome


###  Testing the code

In [462]:
wp1 = src.get_webpage_info('http://www.sohoparish.co.uk/', verbose=True)

About-us or ethos urls: 8
['http://www.sohoparish.co.uk/Introduction-curriculum', 'http://www.sohoparish.co.uk/../Introduction-curriculum', 'http://www.sohoparish.co.uk/britishvalues', 'http://www.sohoparish.co.uk/../../Introduction-curriculum', 'http://www.sohoparish.co.uk/../../School-values-n-goals', 'http://www.sohoparish.co.uk/../../britishvalues', 'http://www.sohoparish.co.uk/../School-values-n-goals', 'http://www.sohoparish.co.uk/../britishvalues']
[u'We follow the Early Years Foundation Stage (EYFS) Curriculum in Reception and the National Curriculum in Key Stage 1 (Years 1 and 2) and Key Stage 2 (Years 3-6).   ', u"Our curriculum is structured in all years through a 'creative curriculum' or 'topic-based' approach.  This means that we select a main topic and teach all subjects in the context of that topic. ", u'We aim to select topics based upon the enthusiasms and interests of the children themselves and consult with the children to develop our curriculum.  Topics change every

In [463]:
wp2 = src.get_webpage_info('http://www.wcsch.com/', verbose=True)

About-us or ethos urls: 2
['http://www.wcsch.com/About-Us', 'http://www.wcsch.com/Prospectus-Request']
[u'Ours is a school with a Christian foundation and character but we welcome boys of all faiths and of no faith. Our strong moral and spiritual ethos ensures pupils develop and prosper during their time at our school.', u'With only 140 boys admitted into each year group, Westminster City School is relatively small. This enables us to get to know our pupils as individuals quickly, teach in small groups and ensure every pupil enjoys coming to school and challenging themselves to be the best they can.', u"The school's hardworking and committed staff deliver the curriculum in a way that ensures all our pupils are able to learn and achieve to the best of their ability. Our GCSE results are well above the national average with pupils achieving results that are in the top third nationally. Class sizes are often under 20 enabling those who need extra support in English and mathematics to rece

In [464]:
wp7 = src.get_webpage_info('http://www.oasisacademysouthbank.org/', verbose=True) 

About-us or ethos urls: 2
['http://www.oasisacademysouthbank.org/content/about-us-27', 'http://www.oasisacademysouthbank.org/content/vision-values-27']
[u'Oasis Academy South Bank opened in September 2013 - a new secondary school serving the local Waterloo community.  It offers a small, safe, learning environment for local children, with Outstanding (Ofsted 2015) educational opportunities where every student can reach their full potential.', u'Oasis South Bank is part of the Oasis global family \xa9 Oasis Charitable Trust 2016\n\t\tOasis Community Learning is a Company Limited by Guarantee registered in England and Wales number 05398529 and an Exempt Charity,\n\t\tRegistered address 75 Westminster Bridge Road, London SE1 7HS  \u2022  Telephone: 020 7921 4200', u'\n\tOasis is a Christian foundation serving all regardless of faith.', u"All young people, regardless of starting point, will 'Climb the STAIRS to Greatness' at Oasis Academy South Bank. Through love, nurture and a rigorous aca

### Scraping JavaScript pages

http://techstonia.com/scraping-with-phantomjs-and-python.html

In [465]:
url = 'http://www.oasisacademysouthbank.org/'

In [186]:
import platform
from selenium import webdriver

# PhantomJS for different operating systems
if platform.system() == 'Windows':  
    PHANTOMJS_PATH = './phantomjs/phantomjs.exe'
elif platform.system() == 'Darwin':  
    PHANTOMJS_PATH = './phantomjs/phantomjs_mac'
else:  
    PHANTOMJS_PATH = './phantomjs/phantomjs_linux'

# here we'll use pseudo browser PhantomJS
browser = webdriver.PhantomJS(PHANTOMJS_PATH)
browser.get(url)
soup = bs.BeautifulSoup(browser.page_source, 'lxml')
browser.quit()

def match_souplinktag(li, links, keywords=src.KEYWORDS):
    """"""
    # getting all the links with href attribute
    if li.has_attr('href'):                
        # separating the last segment of the href
        segment_link = li.get('href')
        if segment_link.endswith('/'):
            segment_link = segment_link.split('/')[-2].lower()
        else:
            segment_link = segment_link.split('/')[-1].lower()
        # comparing href to the keywords
        if any(key.lower() in segment_link for key in keywords):
            found_link = li.get('href')
            if not found_link.startswith('http'):
                found_link = url+found_link
            if found_link not in links:
                links.append(found_link)
    return links

urls_about = []
linktags = soup.find_all('a')
for li in linktags:
    urls_about = match_souplinktag(li, urls_about, keywords=src.KEYWORDS)
                
print urls_about
    

['http://www.oasisacademysouthbank.org//content/about-us-27', 'http://www.oasisacademysouthbank.org//admission-form-Prospectus', 'http://www.oasisacademysouthbank.org//content/vision-values-27']


Test on problematic URLs:

In [309]:
src.get_webpage_info('http://www.rhylprim.camden.sch.uk/', debug=True)

['http://www.rhylprim.camden.sch.uk/School-Info', 'http://www.rhylprim.camden.sch.uk/British-Values', 'http://www.rhylprim.camden.sch.uk/../School-Info', 'http://www.rhylprim.camden.sch.uk/../British-Values']


[u'Together we learn, achieve and succeed',
 u'Rhyl is a harmonious two-form entry school in the heart of Kentish Town, within the Borough of Camden. We have very high expectations for all our pupils and we deliver a diverse and enriching curriculum in surroundings that allow children to feel happy and safe',
 u"Rhyl's commitment to creativity and the arts has been recognized through our achievement of the Arts Mark Gold award. All children have weekly music lessons from our music specialist, they have the opportunity to learn a range of instruments for free and also have the chance to record in our state of the art recording studio.",
 u'At Rhyl we believe passionately in the power of outdoor learning to inspire our children; we have won numerous awards for our outdoor garden which boasts a mini orchard, a secret herb garden and a wood-fired pizza oven.',
 u'We offer a breakfast club which runs from 8am every day. It is open to pupils from Year 2 to Year 6 who can enjoy a delicious an

In [314]:
src.get_webpage_info('http://www.oasisacademysouthbank.org/', debug=True)

['http://www.oasisacademysouthbank.org//content/about-us-27', 'http://www.oasisacademysouthbank.org//content/vision-values-27']


[u'Oasis Academy South Bank opened in September 2013 - a new secondary school serving the local Waterloo community.  It offers a small, safe, learning environment for local children, with Outstanding (Ofsted 2015) educational opportunities where every student can reach their full potential.',
 u'Oasis South Bank is part of the Oasis global family \xa9 Oasis Charitable Trust 2016\n\t\tOasis Community Learning is a Company Limited by Guarantee registered in England and Wales number 05398529 and an Exempt Charity,\n\t\tRegistered address 75 Westminster Bridge Road, London SE1 7HS  \u2022  Telephone: 020 7921 4200',
 u'\n\tOasis is a Christian foundation serving all regardless of faith.']

###  Testing problematic URLs and improving initial prototype

We can handle now JS and have a more versatile code.  All the above cases failes with the first prototype, used on 200 websites. Most are now succesfully scraped except with a few exceptions (see comments in front).

In [412]:
#url = 'http://www.ourladydolours.co.uk/'
#url = 'http://www.oasisacademyjohanna.org/'
#url = 'http://www.burdettcoutts.westminster.lgfl.net/' #hot a school apparently
#url = 'http://www.oasisacademysouthbank.org/'
#url = 'http://www.svpschool-primary.org.uk/'   # JS
#url = 'http://www.sghsprimary.org.uk/' # no connection
#url = 'http://www.southwark-rc-cathedral.org.uk/' # a cathedral
#url = 'http://www.wtwschool.co.uk/'
#url = 'http://www.friars.southwark.sch.uk/' # not responding
#url = 'http://www.stalbanscamden.co.uk/'
#url = 'http://www.pimlicoacademy.org/'
#url = 'http://www.williamdavis.org.uk/'
#url = 'http://www.serviteprimaryschool.co.uk/'
#url = 'http://www.hungerfordschool.com/' # not found
#url = 'http://www.bethnalgreenacademy.co.uk/'
#url = 'http://www.randalcremer.hackney.sch.uk/'
#url = 'http://www.ourladydolours.co.uk/'
#url = 'http://www.columbia.towerhamlets.sch.uk/' # first one where the 'p' cannot be accessed
#url = 'http://www.fox.rbkc.sch.uk/'
#url = 'http://www.stpatricks.camden.sch.uk/'
#url = 'http://www.kentishtown.camden.sch.uk/'
#url = 'http://www.shaftesburypark.wandsworth.sch.uk/'
#url = 'http://www.rhylprim.camden.sch.uk/'
#url = 'http://www.sjbc.wandsworth.sch.uk/'
#url = 'http://www.biglandgreen.towerhamlets.sch.uk/'
#url = 'http://www.westminsteracademy.net/' # website not found in the hosting provider
#url = 'http://www.mayflowerfederation.org.uk/' # website address changed, redirected
url = 'http://www.wappinghigh.org/'

src.get_webpage_info(url, debug=True)

Total links: 167
About-us or ethos urls: 3
['http://www.wappinghigh.org/about-us', 'http://www.wappinghigh.org/prospectus', 'http://www.wappinghigh.org/ethos-and-aims']


[u'Raising aspirations, broadening horizons and ensuring better outcomes for all its students, Wapping High School builds upon the diversity of student backgrounds, providing all students with a truly personalised curriculum to enable enhanced learning and the development of inquisitive minds.',
 u'The school day at Wapping High School is extended to allow all students to participate in a range of exciting and stimulating enrichment activities.',
 u'Wapping and Shadwell are home to eight well \xadsupported primary schools, attended by around 2,000 pupils drawn from a genuinely mixed community. Historically on reaching the age of 11, many of these children had no obvious secondary school to go to. Each year a significant number of families moved out of the area as a result of this lack of provision. In 2012, the parent led Wapping and Shadwell Secondary Education Trust (WSSET) set up Wapping High School, a co\xad-educational free school for children aged 11 - \xad16.',
 u'Headteacher:  

# Generating a DB (first method)

We do a query search on http://www.oasisacademysouthbank.org/ with the following filters:

- Primary and secondary schools.
-  Ofsted ratings: outstanding [1], good [2], require improvement [3], inadequate [4]
- London

In [129]:
search_url = 'https://www.compare-school-performance.service.gov.uk/find-a-school-in-england?radius=3&orderby=&schoollevel=IsPrimary&schoollevel=IsSecondary&ofstedrating=1&ofstedrating=2&ofstedrating=3&ofstedrating=4&searchtype=search-by-location&keywords=Londres%2C+Royaume-Uni&LocationCoordinates=51.5073509%2C-0.12775829999998223&selectionState=1%2C0%2C0'

Getting the metadata:

In [143]:
na_sc, li_sc, sco_sc, add_sc, lev_sc, ty_sc = get_schools_metadata(search_url)

Getting info for page 1 
Getting data for page 2
Getting data for page 3
Getting data for page 4
Getting data for page 5


In [211]:
links_websites_schools, links_csv_schools = get_schools_urls(li_sc)

100%|██████████| 222/222 [04:41<00:00,  1.23s/it]     | 1/222 [00:00<02:13,  1.65it/s]


In [39]:
links_websites_schools[:5]

['http://www.sohoparish.co.uk/',
 'http://www.st-clementdanes.westminster.sch.uk/',
 'http://www.stjosephs.camden.sch.uk/',
 'http://www.stmwschool.org.uk/',
 'http://www.wcsch.com/']

In [40]:
links_csv_schools[:5]

['https://www.compare-school-performance.service.gov.uk//download-school-data?urn=101131',
 'https://www.compare-school-performance.service.gov.uk//download-school-data?urn=101127',
 'https://www.compare-school-performance.service.gov.uk//download-school-data?urn=100041',
 'https://www.compare-school-performance.service.gov.uk//download-school-data?urn=101138',
 'https://www.compare-school-performance.service.gov.uk//download-school-data?urn=138312']

When we can't parse: JS page, weird HTML structure that BS cannot parse or server not responding. For these links we put a None in the pandas DB.

*generate_schools_db* uses all the above functions for scraping the website with the schools information (https://www.compare-school-performance.service.gov.uk/). It returns (and saves to disk) a pandas table with the metadata plus the scraped "about us" information. It handles properly unsuccessful requests (try/except). The "about us" information is scraped based on a set of HTML tags and a list of keywords, which by default is:

['About', 'about', 'about us', 'ABOUT US', 'About Us', '', '', 'Ethos', 'ETHOS', 'Our School', 'OUR SCHOOL']

In [55]:
df = generate_schools_db(search_url, save_csv='./data/schools_db.csv', about_tags=['a'], 
                          keywords=None, text_tags = ['p', 'li', 'ol', 'ul'])

In [41]:
df[:5]

,address,csv,level,name,score,type,website,text
0,"23 Great Windmill Street, London, W1D 7LF",https://www.compare-school-performance.service...,Primary,Soho Parish CofE Primary School,1,Maintained School,http://www.sohoparish.co.uk/,[As a Church of England school our values are ...
1,"Drury Lane, London, WC2B 5SU",https://www.compare-school-performance.service...,Primary,St Clement Danes CofE Primary School,1,Maintained School,http://www.st-clementdanes.westminster.sch.uk/,"[Located in the heart of Covent Garden, London..."
2,"Macklin Street, London, WC2B 5NA",https://www.compare-school-performance.service...,Primary,St Josephs Primary School,1,Maintained School,http://www.stjosephs.camden.sch.uk/,[When parents and children talk about the St....
3,"18 Old Pye Street, London, SW1P 2DG",https://www.compare-school-performance.service...,Primary,"St Matthew's School, Westminster",2,Maintained School,http://www.stmwschool.org.uk/,[St Matthew's School is a thriving one form en...
4,"55 Palace Street, London, SW1E 5HJ",https://www.compare-school-performance.service...,Secondary and 16 to 18,Westminster City School,2,Academy,http://www.wcsch.com/,[Ours is a school with a Christian foundation ...


From these search query we end up with ~100 schools (and their mined text). We lose half of them  improving the websites mining. BeautifulSoup only handles HTML pages.  pages need to be addressed differently. The classic way of just sending the request to the site and parsing the html won’t work as  is not rendered and thus the body of the page is incomplete.

# Generating a first database (second method)

Testing the second method with a short list of URNs obtained  in notebook ``DB_exploratory_analysis``. 

In [415]:
urns = [100322, 100324, 100326, 100328, 100331]

In [429]:
src.get_schools_urls(urns, mode='urn')

['http://www.avonmore.lbhf.sch.uk/',
 'http://www.brackenbury.lbhf.sch.uk/',
 'http://www.milescoverdaleprimary.co.uk/',
 'http://www.floragardens.lbhf.sch.uk/',
 'http://www.kenmont-primary.org/']

We need a list of school URNs from the government databases. We can take this from the CSV databases (let's assume we have such CSV loaded in a DF called dbnew). If we filter out the closed schools we end up with 20551 schools for all UK.

In [432]:
dbnew.shape

(20551, 16)

In [434]:
list_urns = dbnew['URN'].tolist()

In [438]:
dbnew['URN'].to_string('./data/list_urns.txt', index=False)

In [446]:
len(list_urns)

20551

Then we pass the list of URNs to the function  ``get_schools_urls`` to retrieve the URLs .

In [ ]:
list_school_urls = src.get_schools_urls(list_urns, mode'urn')

We obtain a list of school websites. Many of those  are actually missing in the  https://www.compare-school-performance.service.gov.uk/ site so we have a lot of None values.

Finally we  process this list of URLs using ``get_webpage_text_from_url_list``.

In [ ]:
mined_text = src.get_webpage_text_from_url_list(list_school_urls)

It takes a while to mine all the school URLs. So far we've processed the first 7000 entries from the ``list_school_urls`` list.

#  Creating the master DB

We create a  db_master.csv file with 5 columns: URN, NAME, URL, TEXT and OFSTED. URN, NAME and OFSTED come from the https://www.gov.uk/government/statistical-data-sets/monthly-management-information-ofsteds-school-inspections-outcomes January CSV. We joined withe the main CSV in https://www.compare-school-performance.service.gov.uk/download-data and filtered out the closed schools. We end up with 20551 rows. The URL and TEXT are the web scrapped data.

In [1]:
df = pd.read_csv('./data/db_master.csv')

df['rawtext'] = np.nan

import pickle

# loading pickle files with scraped text
text1 = pickle.load(open('./ignored/text1', 'r'))
text2 = pickle.load(open('./ignored/text2', 'r'))
text3 = pickle.load(open('./ignored/text3', 'r'))
text6 = pickle.load(open('./ignored/text6', 'r'))
text7 = pickle.load(open('./ignored/text7', 'r'))
text10 = pickle.load(open('./ignored/text10', 'r'))
text12 = pickle.load(open('./ignored/text12', 'r'))

df['rawtext'][0:1000] = pd.Series(text1, dtype=object)
df['rawtext'][1000:2000] = pd.Series(text2, dtype=object)
df['rawtext'][2000:3000] = pd.Series(text3, dtype=object)
df['rawtext'][5000:6000] = pd.Series(text6, dtype=object)
df['rawtext'][6000:7000] = pd.Series(text7, dtype=object)
df['rawtext'][9000:10000] = pd.Series(text10, dtype=object)
df['rawtext'][11000:12000] = pd.Series(text12, dtype=object)

df.to_pickle('./data/db_master.pickle')